In [1]:

import json
users = json.load(open("/home/jovyan/shares/SR004.nfs2/amaksimova/exp/ranker/data/users.json"))
vacancy = json.load(open("/home/jovyan/shares/SR004.nfs2/amaksimova/exp/ranker/data/vacancy.json"))


In [2]:
import pandas as pd
rs=pd.read_csv("/home/jovyan/shares/SR004.nfs2/amaksimova/exp/ranker/data/relavence_scores.csv")
rs

,Unnamed: 0,vacancy_id,user_id,relavence_score
0,0,v_2565203,u_403016,0.33
1,1,v_2459679,u_1099800,0.33
2,2,v_2493588,u_1099800,0.33
3,3,v_612324,u_1099800,0.33
4,4,v_529585,u_1099800,0.33
...,...,...,...,...
48773,48773,v_1902404,u_302844,0.33
48774,48774,v_56015,u_302844,0.33
48775,48775,v_374342,u_302844,0.33
48776,48776,v_1013185,u_441021,0.33


In [18]:
users

{'u_403016': {'vacancy_id': ['v_2565203'], 'action_type': [2]},
 'u_1099800': {'vacancy_id': ['v_2459679',
   'v_2493588',
   'v_612324',
   'v_529585',
   'v_2493588',
   'v_1458341',
   'v_1135877',
   'v_2559034',
   'v_782232',
   'v_2082486',
   'v_423474',
   'v_826996',
   'v_782232',
   'v_1956729',
   'v_2082486',
   'v_2456123',
   'v_166186',
   'v_2302169',
   'v_1995396',
   'v_2412691'],
  'action_type': [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]},
 'u_588270': {'vacancy_id': ['v_165588'], 'action_type': [2]},
 'u_706637': {'vacancy_id': ['v_1012981'], 'action_type': [2]},
 'u_513709': {'vacancy_id': ['v_968261', 'v_968261', 'v_968261'],
  'action_type': [2, 2, 1]},
 'u_547480': {'vacancy_id': ['v_1275435', 'v_1275435'], 'action_type': [1, 2]},
 'u_21459': {'vacancy_id': ['v_520355', 'v_730882', 'v_644141'],
  'action_type': [2, 2, 2]},
 'u_1135569': {'vacancy_id': ['v_2472553', 'v_809650', 'v_2433492'],
  'action_type': [2, 2, 2]},
 'u_1070396': {'vacan

In [1]:
vacancy

NameError: name 'vacancy' is not defined

In [7]:
features = [
            'USER AVG COS SIM FOR LIKED RESPOND VACANCY NAME',
            'USER keySkills coverage',
            'USER AVG SALARY FROM',
            'USER AVG SALARY TO',
            'VACANCY SALARY FROM',
            'VACANCY SALARY TO']

{'u_349273': {'vacancy_id': ['v_343228',
   'v_118609',
   'v_118609',
   'v_791114',
   'v_1394380'],
  'action_type': [2, 2, 2, 2, 2]},
 'u_611189': {'vacancy_id': ['v_1566766',
   'v_1396727',
   'v_1980488',
   'v_1980488',
   'v_2531806',
   'v_1396727'],
  'action_type': [2, 2, 1, 2, 2, 1]},
 'u_436978': {'vacancy_id': ['v_1920480'], 'action_type': [2]},
 'u_416342': {'vacancy_id': ['v_6224'], 'action_type': [2]},
 'u_120860': {'vacancy_id': ['v_2498654', 'v_2585851', 'v_2000041'],
  'action_type': [2, 2, 2]},
 'u_546693': {'vacancy_id': ['v_2642292', 'v_236844'], 'action_type': [2, 2]},
 'u_460996': {'vacancy_id': ['v_1861136',
   'v_154518',
   'v_2267812',
   'v_124615',
   'v_1960729',
   'v_1645662',
   'v_2267812',
   'v_701983',
   'v_1438760',
   'v_1756551',
   'v_701983',
   'v_2696844',
   'v_2023779',
   'v_687152',
   'v_91454',
   'v_2594818',
   'v_884933'],
  'action_type': [2, 2, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2]},
 'u_1012470': {'vacancy_id': ['v_1723

In [16]:
#rs = rs.reset_index()
coverage = []
vacancy_salary_from = []
vacancy_salary_to = []
user_salary_from = []
user_salary_to = []
intersection = []
import numpy as np
from tqdm import tqdm
for index, row in tqdm(rs.iterrows()):
    
    vacancy_id = row['vacancy_id']
    vacancy_row = vacancy[vacancy_id]
    user_row = users[row['user_id']]

    vacancy_salary_to.append(vacancy_row['compensation.to'])
    vacancy_salary_from.append(vacancy_row['compensation.from'])
    
    user_salary_to_t = []
    user_salary_from_t = []
    ind = -1
    
    skills_v = set(vacancy_row['keySkills'])
    skills_u = set()

    if vacancy_id in user_row['vacancy_id']:
        ind = user_row['vacancy_id'].index(vacancy_id)

    for i in range(len(user_row['action_type'])):
        if i!=ind and user_row['action_type'][i]!=2:
            if vacancy[user_row['vacancy_id'][i]]['compensation.to'] is not None:
                user_salary_to_t.append(vacancy[user_row['vacancy_id'][i]]['compensation.to'])
            if vacancy[user_row['vacancy_id'][i]]['compensation.from'] is not None:
                user_salary_from_t.append(vacancy[user_row['vacancy_id'][i]]['compensation.from'])
            ss = vacancy[user_row['vacancy_id'][i]]['keySkills']
            for i in ss:
                skills_u.add(i)

    intersection_l=len(set.intersection(skills_u, skills_v))
    aaa = 0
    if intersection_l:
        aaa = len(skills_v) / intersection_l
    intersection.append(aaa)

    
    if len(user_salary_to_t):
        user_salary_to.append(np.nanmean(user_salary_to_t))
    else:
        user_salary_to.append(0)
    if len(user_salary_from_t):
        user_salary_from.append(np.nanmean(user_salary_from_t))
    else:
        user_salary_from.append(0)



30585it [00:02, 13004.05it/s]


In [11]:
rs = pd.read_csv("/home/jovyan/shares/SR004.nfs2/amaksimova/exp/ranker/data/df.csv")

In [12]:
rs['user_salary_from'] = user_salary_from
rs['user_salary_to'] = user_salary_to
rs['vacancy_salary_from'] = vacancy_salary_from
rs['vacancy_salary_to'] = vacancy_salary_to
rs['intersection_keyskill'] = intersection
rs

,Unnamed: 0.1,index,Unnamed: 0,vacancy_id,user_id,relavence_score,mean_names,mean_desc,user_salary_from,user_salary_to,vacancy_salary_from,vacancy_salary_to,intersection_keyskill
0,0,0,0,v_2565203,u_403016,0.33,0.000000,0.000000,0.000000,0.0,NaN,NaN,0.000000
1,1,1,1,v_2459679,u_1099800,0.33,0.000000,0.000000,0.000000,0.0,86200.0,154820.0,0.000000
2,2,2,2,v_2493588,u_1099800,0.33,0.000000,0.000000,0.000000,0.0,74000.0,NaN,0.000000
3,3,3,3,v_612324,u_1099800,0.33,0.000000,0.000000,0.000000,0.0,75000.0,NaN,0.000000
4,4,4,4,v_529585,u_1099800,0.33,0.000000,0.000000,0.000000,0.0,100000.0,NaN,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
48773,48773,48773,48773,v_1902404,u_302844,0.33,0.581152,0.666719,39722.222222,45875.0,25000.0,NaN,1.166667
48774,48774,48774,48774,v_56015,u_302844,0.33,0.404525,0.631718,39722.222222,45875.0,NaN,25000.0,0.000000
48775,48775,48775,48775,v_374342,u_302844,0.33,0.422278,0.664646,39722.222222,45875.0,13000.0,33800.0,0.000000
48776,48776,48776,48776,v_1013185,u_441021,0.33,0.000000,0.000000,0.000000,0.0,NaN,NaN,0.000000


In [14]:
rs = rs[rs['mean_names']>0]
rs

,Unnamed: 0.1,index,Unnamed: 0,vacancy_id,user_id,relavence_score,mean_names,mean_desc,user_salary_from,user_salary_to,vacancy_salary_from,vacancy_salary_to,intersection_keyskill
23,23,23,23,v_968261,u_513709,0.33,1.000000,1.000000,70000.000000,0.0,70000.0,NaN,1.000000
24,24,24,24,v_968261,u_513709,0.33,1.000000,1.000000,70000.000000,0.0,70000.0,NaN,1.000000
25,25,25,25,v_968261,u_513709,1.00,1.000000,1.000000,70000.000000,0.0,70000.0,NaN,1.000000
42,42,42,42,v_128856,u_1070284,0.33,1.000000,1.000000,0.000000,0.0,NaN,NaN,1.000000
43,43,43,43,v_128856,u_1070284,1.00,1.000000,1.000000,0.000000,0.0,NaN,NaN,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
48771,48771,48771,48771,v_915763,u_302844,0.66,0.519392,0.691904,39722.222222,45875.0,37500.0,46000.0,1.000000
48772,48772,48772,48772,v_1726400,u_302844,0.33,0.369177,0.653639,39722.222222,45875.0,46000.0,69000.0,1.666667
48773,48773,48773,48773,v_1902404,u_302844,0.33,0.581152,0.666719,39722.222222,45875.0,25000.0,NaN,1.166667
48774,48774,48774,48774,v_56015,u_302844,0.33,0.404525,0.631718,39722.222222,45875.0,NaN,25000.0,0.000000


In [15]:
rs.to_csv("train_data.csv")

In [15]:
rs

,Unnamed: 0,vacancy_id,user_id,relavence_score,mean_names,mean_desc
0,0,v_343228,u_349273,0.33,0.000000,0.000000
1,1,v_118609,u_349273,0.33,0.000000,0.000000
2,2,v_118609,u_349273,0.33,0.000000,0.000000
3,3,v_791114,u_349273,0.33,0.000000,0.000000
4,4,v_1394380,u_349273,0.33,0.000000,0.000000
5,5,v_1566766,u_611189,0.33,0.544062,0.563319
6,6,v_1396727,u_611189,0.33,0.692364,0.702313
7,7,v_1980488,u_611189,1.00,0.384727,0.404627
8,8,v_1980488,u_611189,0.33,0.384727,0.404627
9,9,v_2531806,u_611189,0.33,0.692364,0.586833
